# ANN Churn Prediction

## Importing dependencies

In [6]:
import pandas as pd  
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
import pickle

## Importing dataset

In [35]:
df = pd.read_csv("dataset/Churn_Modelling.csv")
df.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


## Preprocessing

- RowNumber, CustomerId, Surname are non essential features

In [36]:
df.drop(["RowNumber", "CustomerId", "Surname"],axis=1, inplace=True)

In [37]:
df.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [38]:
df["Geography"].unique()

array(['France', 'Spain', 'Germany'], dtype=object)

- Geography & Gender are categorical variable, covert it into numerical variable. 

In [39]:
label_encoder_gender = LabelEncoder()

df['Gender'] = label_encoder_gender.fit_transform(df['Gender'])
df.head() 

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,0,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,0,41,1,83807.86,1,0,1,112542.58,0
2,502,France,0,42,8,159660.80,3,1,0,113931.57,1
3,699,France,0,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,0,43,2,125510.82,1,1,1,79084.10,0


In [40]:
from sklearn.preprocessing import OneHotEncoder
encoder_geography = OneHotEncoder(sparse_output=False)

onehot_encoded_geography = encoder_geography.fit_transform(df[['Geography']])
print(onehot_encoded_geography)
print(type(onehot_encoded_geography))

[[1. 0. 0.]
 [0. 0. 1.]
 [1. 0. 0.]
 ...
 [1. 0. 0.]
 [0. 1. 0.]
 [1. 0. 0.]]
<class 'numpy.ndarray'>


In [41]:
print(encoder_geography.get_feature_names_out(['Geography']))

['Geography_France' 'Geography_Germany' 'Geography_Spain']


In [42]:
geo_df = pd.DataFrame(onehot_encoded_geography, columns=encoder_geography.get_feature_names_out(['Geography']))
print(geo_df)

      Geography_France  Geography_Germany  Geography_Spain
0                  1.0                0.0              0.0
1                  0.0                0.0              1.0
2                  1.0                0.0              0.0
3                  1.0                0.0              0.0
4                  0.0                0.0              1.0
...                ...                ...              ...
9995               1.0                0.0              0.0
9996               1.0                0.0              0.0
9997               1.0                0.0              0.0
9998               0.0                1.0              0.0
9999               1.0                0.0              0.0

[10000 rows x 3 columns]


In [44]:
df_final = pd.concat([df.drop(['Geography'], axis=1), geo_df], axis=1)

df_final.head()


,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0


In [45]:
temp = df_final['Exited']
df_final = pd.concat([df_final.drop(['Exited'], axis=1), temp], axis=1)
df_final.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain,Exited
0,619,0,42,2,0.00,1,1,1,101348.88,1.0,0.0,0.0,1
1,608,0,41,1,83807.86,1,0,1,112542.58,0.0,0.0,1.0,0
2,502,0,42,8,159660.80,3,1,0,113931.57,1.0,0.0,0.0,1
3,699,0,39,1,0.00,2,0,0,93826.63,1.0,0.0,0.0,0
4,850,0,43,2,125510.82,1,1,1,79084.10,0.0,0.0,1.0,0


### Saving encoders 

In [46]:
with open('models/label_encoder_gender', 'wb') as file:
    pickle.dump(label_encoder_gender,file) 
    
with open('models/one_hot_encoder_geography', 'wb') as file:
    pickle.dump(encoder_geography, file)

## Independent & Dependent Feature splitting

In [48]:
X = df_final.iloc[:,:-1]
y = df_final.iloc[:,-1]

## Train test split

In [53]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2, random_state=42, stratify=y)

In [54]:
print(f"X_train: ${X_train.shape}")
print(f"X_test: ${X_test.shape}")
print(f"y_train: ${y_train.shape}")
print(f"y_test: ${y_test.shape}")

X_train: $(8000, 12)
X_test: $(2000, 12)
y_train: $(8000,)
y_test: $(2000,)


## Scaling Features

In [55]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [56]:
with open('models/scaler.pkl', 'wb') as file:
    pickle.dump(scaler, file)